# MiddleWare: Human In The Loop

## Setup

In [1]:
from langchain_community.utilities import SQLDatabase

db = SQLDatabase.from_uri("sqlite:///Chinook.db")

In [3]:
from langchain.chat_models import init_chat_model

model = init_chat_model("openai:gpt-5")

In [4]:
from typing import TypedDict

# Setup your agent runtime context
class Context(TypedDict):
    first: str
    last: str
    

In [5]:
import re

from langgraph.runtime import Runtime, get_runtime
from langchain_core.tools import tool

def _execute_sql(query: str) -> str:
    """Execute a SQLite command and return results.
    Named parameters like :first and :last will be filled from runtime context."""
    runtime = get_runtime(Context)
    try:
        placeholders = re.findall(r":(\w+)", query)

        if placeholders:
            params = {
                p: runtime.context[p]
                for p in placeholders
                if p in runtime.context
            }
            return db.run(query, parameters=params)
        else:
            return db.run(query)
    except Exception as e:
        return f"Error: {e}"
@tool
def execute_nonemployee_sql(query: str) -> str:
    """Execute a SQLite command and return results.
    Call this for any access that do not access the employee table.
    Named parameters like :first and :last will be filled from runtime context."""
    return _execute_sql(query)

@tool
def execute_employee_sql(query: str) -> str:
    """Execute a SQLite command that access the employee table and return results.
    Named parameters like :first and :last will be filled from runtime context."""
    return _execute_sql(query)



In [6]:
SYSTEM = """You are a careful SQLite analyst.

Rules:
- Think step-by-step.
- When you need data, call the tool `execute_sql` with ONE SELECT query.
- Read-only only; no INSERT/UPDATE/DELETE/ALTER/DROP/CREATE/REPLACE/TRUNCATE.
- Limit to 5 rows unless the user explicitly asks otherwise.
- If the tool returns 'Error:', revise the SQL and try again.
- Prefer explicit column lists; avoid SELECT *.
- If a query involves a specific customer, always use the named parameters
  :first and :last (instead of hard-coding values).
  Example: SELECT InvoiceId, Total FROM Invoice
           JOIN Customer ON Invoice.CustomerId = Customer.CustomerId
           WHERE Customer.FirstName = :first AND Customer.LastName = :last;
  Assume the customer first and last name are known and will be inserted into the query later.
"""

In [8]:
from langchain.agents.middleware import HumanInTheLoopMiddleware 
from langchain.agents import create_agent
from langchain_core.messages import SystemMessage
from langgraph.checkpoint.memory import InMemorySaver

llm = init_chat_model("gpt-5", model_provider="openai")

agent = create_agent(
    model=model,
    tools=[execute_employee_sql, execute_nonemployee_sql],
    prompt=SYSTEM,
    checkpointer=InMemorySaver(),
    context_schema=Context,
    middleware=[
        HumanInTheLoopMiddleware( 
            interrupt_on={
                "execute_employee_sql": {"allow_accept": True, "allow_respond": True},  # No editing allowed
            },
            description_prefix="Tool execution pending approval",
        ),
    ],

)



In [9]:
question = "What was the total on my last invoice?"

result = agent.invoke(
    {"messages": [{"role": "user", "content": question}]},
    context= {"first": "Frank", "last": "Harris"},
    config = {"configurable": {"thread_id": "1"}} 
)
print(result["messages"][-1].content)

Your last invoice total was 5.94 (Invoice 374 on 2013-07-04).


In [10]:
from langgraph.types import Command
question = "What are the names of all the employees?"

config = {"configurable": {"thread_id": "1"}} 

result = agent.invoke(
    {"messages": [{"role": "user", "content": question}]},
    context= {"first": "Frank", "last": "Harris"},
    config=config
)
if '__interrupt__' in result:
    print(f"\033[1;3;31m{80*'-'}\033[0m")
    print(f"\033[1;3;31m Interrupt:{result['__interrupt__'][-1].value[-1]['description']}\033[0m")

    result = agent.invoke(
        Command( 
            resume=[{"type": "accept"}]  # or "edit", "respond"
        ), 
        config=config # Same thread ID to resume the paused conversation
    )
    print(f"\033[1;3;31m{80*'-'}\033[0m")
           
print(result["messages"][-1].content)

--------------------------------------------------------------------------------
 Interrupt:Tool execution pending approval

Tool: execute_employee_sql
Args: {'query': 'SELECT e.FirstName, e.LastName\nFROM Employee AS e\nORDER BY e.LastName, e.FirstName;'}
--------------------------------------------------------------------------------
Here are the employee names (last name, first name order):
- Adams, Andrew
- Callahan, Laura
- Edwards, Nancy
- Johnson, Steve
- King, Robert
- Mitchell, Michael
- Park, Margaret
- Peacock, Jane
